In [1]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [6]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [34]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
from datetime import datetime  # Importa a classe 'datetime' do módulo 'datetime' para registrar a data e hora.
import random  # Importa o módulo 'random' para gerar números aleatórios e fazer escolhas aleatórias.
import time    # Importa o módulo 'time' para controlar o tempo, como pausas.
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, HTML, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [8]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [9]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# **INICIO DO MEU AGENTE**

In [10]:
###############################################
# --- ARMAZENA DADOS DO USUÁRIO NA SESSÃO --- #
###############################################
historico_desempenho = {}  # Cria um dicionário vazio chamado 'historico_desempenho' para armazenar o desempenho dos usuários. As chaves serão os IDs dos usuários e os valores serão listas de seus resultados.
usuario_atual = "usuario_teste"  # Define uma variável 'usuario_atual' com um ID de usuário para simular um usuário único por enquanto.

In [11]:
################################################################
# Função de Exercício de Memória de Palavras (Tempo Aumentado)
################################################################
def exercicio_memoria_palavras(num_palavras=5, duracao_apresentacao=5): # Aumentado para 5 segundos
    palavras = ["casa", "árvore", "livro", "sol", "lua", "carro", "flor", "rio", "pássaro", "mesa", "janela", "porta", "cadeira", "papel", "caneta"]
    palavras_sorteadas = random.sample(palavras, num_palavras)
    print(f"\nMemorize as seguintes {num_palavras} palavras em {duracao_apresentacao} segundos:")
    print(palavras_sorteadas)
    time.sleep(duracao_apresentacao)
    print("\nTempo esgotado!")
    resposta_usuario = input("Digite as palavras que você lembra, separadas por espaço: ").split()
    acertos = 0
    palavras_corretas = []
    palavras_faltantes = []
    for palavra in resposta_usuario:
        if palavra in palavras_sorteadas and palavra not in palavras_corretas:
            acertos += 1
            palavras_corretas.append(palavra)
    for palavra in palavras_sorteadas:
        if palavra not in palavras_corretas:
            palavras_faltantes.append(palavra)
    print(f"\nVocê lembrou de {acertos} de {num_palavras} palavras.")
    print("Palavras lembradas:", palavras_corretas)
    print("Palavras que faltaram:", palavras_faltantes)
    return {"palavras_apresentadas": palavras_sorteadas,
            "resposta_usuario": resposta_usuario,
            "acertos": acertos,
            "total_palavras": num_palavras,
            "acertou": (acertos == num_palavras)}

In [14]:
#################################################################
# Função de Exercício de Atenção Ímpar (Sem Alteração de Tempo) #
#################################################################
def exercicio_atencao_impar(tamanho_sequencia=7):
    sequencia = []
    posicao_impar = random.randint(0, tamanho_sequencia - 1)
    numero_impar = random.choice([1, 3, 5, 7, 9])
    for i in range(tamanho_sequencia):
        if i == posicao_impar:
            sequencia.append(numero_impar)
        else:
            sequencia.append(random.choice([2, 4, 6, 8]))
    print("\nEncontre o número ímpar na seguinte sequência:")
    print(sequencia)
    resposta_usuario_str = input("Digite o número ímpar encontrado: ")
    try:
        resposta_usuario = int(resposta_usuario_str)
        acertou = (resposta_usuario == numero_impar)
    except ValueError:
        acertou = False
    if acertou:
        print("Parabéns! Você acertou.")
    else:
        print(f"Resposta incorreta. O número ímpar era {numero_impar}.")
    return {"sequencia_apresentada": sequencia,
            "resposta_usuario": resposta_usuario_str,
            "numero_impar_correto": numero_impar,
            "acertou": acertou}

In [15]:
#############################################################################
# Função de Exercício de Linguagem Completar Frase (Sem Alteração de Tempo) #
#############################################################################
def exercicio_linguagem_completar_frase():
    frases = [
        {"frase": "O gato subiu na ___.", "resposta": "mesa"},
        {"frase": "Eu gosto de comer ___ no café da manhã.", "resposta": "pão"},
        {"frase": "O céu está muito ___ hoje.", "resposta": "azul"},
        {"frase": "Preciso comprar um novo ___ para escrever.", "resposta": "livro"},
        {"frase": "A flor mais bonita do jardim é a ___.", "resposta": "rosa"}
    ]
    frase_escolhida = random.choice(frases)
    frase_com_vazio = frase_escolhida["frase"].replace("___", "_____")
    print(f"\nComplete a seguinte frase:")
    print(frase_com_vazio)
    resposta_usuario = input("Digite a palavra que falta: ").strip().lower()
    acertou = (resposta_usuario == frase_escolhida["resposta"])
    if acertou:
        print("Parabéns! Você completou a frase corretamente.")
    else:
        print(f"Resposta incorreta. A palavra correta era: {frase_escolhida['resposta']}.")
    return {"frase_apresentada": frase_com_vazio,
            "palavra_correta": frase_escolhida["resposta"],
            "resposta_usuario": resposta_usuario,
            "acertou": acertou}

In [16]:
################################################################
# Função de Exercício de Memória de Números (Tempo Aumentado). #
################################################################
def exercicio_memoria_numeros(tamanho_sequencia=6, duracao_apresentacao=5): # Aumentado para 5 segundos
    sequencia = [random.randint(0, 9) for _ in range(tamanho_sequencia)]
    print(f"\nMemorize a seguinte sequência de {tamanho_sequencia} números em {duracao_apresentacao} segundos:")
    print(sequencia)
    time.sleep(duracao_apresentacao)
    print("\nTempo esgotado!")
    resposta_usuario_str = input("Digite a sequência de números que você lembra, separada por espaço: ")
    try:
        resposta_usuario = [int(x) for x in resposta_usuario_str.split()]
        acertou = (resposta_usuario == sequencia)
    except ValueError:
        acertou = False
    if acertou:
        print("Parabéns! Você acertou a sequência.")
    else:
        print(f"Resposta incorreta. A sequência correta era: {sequencia}.")
    return {"sequencia_apresentada": sequencia,
            "resposta_usuario": resposta_usuario_str, # Mantido como string no retorno para registro
            "acertou": acertou}

In [41]:
################################################################################
# Função de Exercício de Atenção Contagem de Letras (Sem Alteração de Tempo).  #
################################################################################
def exercicio_atencao_contagem_letras(tamanho_sequencia=15, letra_alvo='a'):
    letras = [random.choice('abcdefg') for _ in range(tamanho_sequencia)]
    # Calcula a contagem correta fora do bloco try
    contagem_correta = letras.count(letra_alvo)

    print(f"\nConte quantas vezes a letra '{letra_alvo}' aparece na seguinte sequência:")
    print(" ".join(letras))
    resposta_usuario_str = input("Digite o número de vezes que a letra aparece: ")

    try:
        resposta_usuario = int(resposta_usuario_str)
        acertou = (resposta_usuario == contagem_correta)
    except ValueError:
        # Se a entrada não for um inteiro válido, está incorreta
        acertou = False
        # Você pode querer definir um valor padrão para resposta_usuario_str
        # se quiser registrar a entrada inválida no histórico,
        # mas ela já é capturada como resposta_usuario_str no retorno.

    if acertou:
        print("Parabéns! Você acertou.")
    else:
        # contagem_correta agora tem garantia de ter um valor
        print(f"Resposta incorreta. A letra '{letra_alvo}' aparece {contagem_correta} vezes.")

    return {"sequencia_apresentada": letras,
            "letra_alvo": letra_alvo,
            "resposta_usuario": resposta_usuario_str, # Mantido como string para registro
            "contagem_correta": contagem_correta,
            "acertou": acertou}

In [19]:
#######################################################################
#     PARA INICIAR OS EXERCÍCIOS COGNITIVOS, EXECUTE ESSA CÉLULA.     #
# Função de Exercício de Linguagem Sinônimo (Sem Alteração de Tempo). #
#######################################################################
def exercicio_linguagem_sinonimo():
    palavras_sinonimos = {
        "feliz": ["alegre", "contente", "risonho"],
        "triste": ["melancólico", "abatido", "deprimido"],
        "grande": ["enorme", "imenso", "gigantesco"],
        "pequeno": ["minúsculo", "diminuto", "liliputiano"],
        "rápido": ["veloz", "ligeiro", "ágil"]
    }
    palavra_chave = random.choice(list(palavras_sinonimos.keys()))
    sinonimo_correto = random.choice(palavras_sinonimos[palavra_chave])
    opcoes = random.sample(palavras_sinonimos[palavra_chave] + random.sample(list(palavras_sinonimos.keys()), 2), 4)
    if sinonimo_correto not in opcoes:
        opcoes[random.randint(0, 3)] = sinonimo_correto
    random.shuffle(opcoes)

    print(f"\nQual o sinônimo da palavra '{palavra_chave}'?")
    for i, opcao in enumerate(opcoes):
        print(f"{i+1}. {opcao}")

    resposta_usuario_str = input("Digite o número da opção correta: ")
    try:
        resposta_usuario_index = int(resposta_usuario_str) - 1
        if 0 <= resposta_usuario_index < 4:
            resposta_usuario = opcoes[resposta_usuario_index]
            acertou = (resposta_usuario == sinonimo_correto)
        else:
            acertou = False
            resposta_usuario = "Opção inválida"
    except ValueError:
        acertou = False
        resposta_usuario = "Entrada inválida"

    if acertou:
        print("Parabéns! Você acertou.")
    else:
        print(f"Resposta incorreta. O sinônimo correto era: {sinonimo_correto}.")

    return {"palavra_chave": palavra_chave,
            "sinonimo_correto": sinonimo_correto,
            "opcoes": opcoes,
            "resposta_usuario": resposta_usuario,
            "acertou": acertou}

In [20]:
##############################################################
# Função de Exercício de Memória de Cores (Tempo Aumentado). #
##############################################################
def exercicio_memoria_cores(num_cores=4, duracao_apresentacao=5): # Aumentado para 5 segundos
    cores = ["vermelho", "azul", "verde", "amarelo", "laranja", "roxo", "branco", "preto"]
    cores_sorteadas = random.sample(cores, num_cores)
    print(f"\nMemorize a seguinte sequência de {num_cores} cores em {duracao_apresentacao} segundos:")
    print(cores_sorteadas)
    time.sleep(duracao_apresentacao)
    print("\nTempo esgotado!")
    resposta_usuario_str = input("Digite as cores que você lembra, separadas por espaço: ")
    resposta_usuario = resposta_usuario_str.split() # Garante que resposta_usuario seja definida aqui
    acertos = 0
    cores_corretas = []
    cores_faltantes = []
    for cor in resposta_usuario:
        if cor in cores_sorteadas and cor not in cores_corretas:
            acertos += 1
            cores_corretas.append(cor)
    for cor in cores_sorteadas:
        if cor not in cores_corretas:
            cores_faltantes.append(cor)
    print(f"\nVocê lembrou de {acertos} de {num_cores} cores.")
    print("Cores lembradas:", cores_corretas)
    print("Cores que faltaram:", cores_faltantes)
    return {"cores_apresentadas": cores_sorteadas,
            "resposta_usuario": resposta_usuario_str, # Salva como string para registro
            "acertos": acertos,
            "total_cores": num_cores,
            "acertou": (acertos == num_cores)}

In [21]:
####################################################################
# Função de Exercício de Atenção Símbolos (Sem Alteração de Tempo) #
####################################################################
def exercicio_atencao_simbolos(tamanho_sequencia=12, simbolo_alvo='*'):
    simbolos = ['*', '#', '$', '%', '&', '@']
    sequencia = [random.choice(simbolos) for _ in range(tamanho_sequencia)]
    print(f"\nConte quantas vezes o símbolo '{simbolo_alvo}' aparece na seguinte sequência:")
    print(" ".join(sequencia))
    resposta_usuario_str = input("Digite o número de vezes que o símbolo aparece: ")
    try:
        resposta_usuario = int(resposta_usuario_str)
        contagem_correta = sequencia.count(simbolo_alvo)
        acertou = (resposta_usuario == contagem_correta)
    except ValueError:
        acertou = False
    if acertou:
        print("Parabéns! Você acertou.")
    else:
        print(f"Resposta incorreta. O símbolo '{simbolo_alvo}' aparece {contagem_correta} vezes.")
    return {"sequencia_apresentada": sequencia,
            "simbolo_alvo": simbolo_alvo,
            "resposta_usuario": resposta_usuario_str,
            "contagem_correta": contagem_correta,
            "acertou": acertou}

In [22]:
#######################################################################
# Função de Exercício de Linguagem Anagrama (Sem Alteração de Tempo). #
#######################################################################
def exercicio_linguagem_anagrama():
    palavras = ["tempo", "livro", "amigo", "terra", "chuva", "noite"]
    palavra_original = random.choice(palavras)
    anagrama = "".join(random.sample(palavra_original, len(palavra_original)))
    print(f"\nDecifre o seguinte anagrama: {anagrama}")
    resposta_usuario = input("Digite a palavra original: ").strip().lower()
    acertou = (resposta_usuario == palavra_original)
    if acertou:
        print("Parabéns! Você decifrou o anagrama.")
    else:
        print(f"Resposta incorreta. A palavra original era: {palavra_original}.")
    return {"anagrama": anagrama,
            "palavra_original": palavra_original,
            "resposta_usuario": resposta_usuario,
            "acertou": acertou}

In [23]:
#############################################################
# Função de Exercício de Memória Visual (Tempo Aumentado).  #
#############################################################
def exercicio_memoria_visual(num_elementos=5, duracao_apresentacao=5): # Aumentado para 5 segundos
    elementos = ["O quadrado vermelho", "O círculo azul", "O triângulo verde", "A estrela amarela", "O losango roxo"]
    elementos_sorteados = random.sample(elementos, num_elementos)
    print(f"\nMemorize a seguinte sequência de {num_elementos} elementos visuais em {duracao_apresentacao} segundos:")
    for elemento in elementos_sorteados:
        print(f"- {elemento}")
        time.sleep(duracao_apresentacao / num_elementos)
    print("\nTempo esgotado!")
    # Chame split apenas uma vez aqui:
    resposta_usuario_list = input("Digite os elementos que você lembra na ordem, separados por ponto e vírgula: ").split(";")
    # Itere sobre a lista resultante:
    resposta_usuario = [resp.strip().lower() for resp in resposta_usuario_list if resp.strip()] # Remove espaços extras, ignora entradas vazias e converte para minúsculas
    #[resp.strip() for resp in resposta_usuario_list if resp.strip()] # Remove espaços extras e ignora entradas vazias

    acertos = 0
    for i in range(min(num_elementos, len(resposta_usuario))):
        if resposta_usuario[i] == elementos_sorteados[i]:
            acertos += 1
    print(f"\nVocê lembrou de {acertos} de {num_elementos} elementos na ordem correta.")
    return {"elementos_apresentados": elementos_sorteados,
            "resposta_usuario": ";".join(resposta_usuario), # Salva como string separada por ponto e vírgula para registro, para manter o formato de entrada
            "acertos": acertos,
            "total_elementos": num_elementos,
            "acertou": (acertos == num_elementos)}

In [24]:
#########################################################################
# Função de Exercício de Atenção Cancelamento (Sem Alteração de Tempo). #
#########################################################################
def exercicio_atencao_cancelamento(grade_size=5, alvo='b'):
    letras = [random.choice('abcdefg') for _ in range(grade_size * grade_size)]
    grade = [letras[i:i + grade_size] for i in range(0, len(letras), grade_size)]
    num_alvos = sum(row.count(alvo) for row in grade)
    print(f"\nMarque (mentalmente ou anote) todas as letras '{alvo}' na seguinte grade:")
    for row in grade:
        print(" ".join(row))
    resposta_usuario_str = input(f"Quantas letras '{alvo}' você encontrou? ")
    try:
        resposta_usuario = int(resposta_usuario_str)
        acertou = (resposta_usuario == num_alvos)
    except ValueError:
        acertou = False
    if acertou:
      print("Parabéns! Você acertou.")
    else:
        print(f"Resposta incorreta. Havia {num_alvos} letras '{alvo}'.")
    return {"grade_apresentada": [ "".join(row) for row in grade],
            "alvo": alvo,
            "resposta_usuario": resposta_usuario_str,
            "contagem_correta": num_alvos,
            "acertou": acertou}

In [25]:
####################################################
# Função de Exercício de Linguagem Categorização   #
####################################################
def exercicio_linguagem_categorizacao():
    palavras = ["maçã", "banana", "carro", "ônibus", "uva", "moto", "pera", "bicicleta"]
    categorias = {"frutas": ["maçã", "banana", "uva", "pera"], "transportes": ["carro", "ônibus", "moto", "bicicleta"]}
    palavras_sorteadas = random.sample(palavras, 6)
    print("\nCategorize as seguintes palavras em 'frutas' e 'transportes':")
    print(palavras_sorteadas)
    resposta_usuario_frutas = [p.strip().lower() for p in input("Digite as frutas separadas por espaço: ").split() if p.strip()] # Processa entrada de frutas
    resposta_usuario_transportes = [p.strip().lower() for p in input("Digite os transportes separados por espaço: ").split() if p.strip()] # Processa entrada de transportes

    acertos_frutas = sum(1 for palavra in resposta_usuario_frutas if palavra in categorias["frutas"])
    acertos_transportes = sum(1 for palavra in resposta_usuario_transportes if palavra in categorias["transportes"])

    # Contagem total correta baseada nas palavras sorteadas e categorias
    total_frutas_sorteadas = sum(1 for palavra in palavras_sorteadas if palavra in categorias["frutas"])
    total_transportes_sorteadas = sum(1 for palavra in palavras_sorteadas if palavra in categorias["transportes"])

    # Acerto total se o usuário categorizou todas as palavras sorteadas corretamente nas listas
    acertou_totalmente = (acertos_frutas == total_frutas_sorteadas) and (acertos_transportes == total_transportes_sorteadas) and \
                         (len(resposta_usuario_frutas) == total_frutas_sorteadas) and (len(resposta_usuario_transportes) == total_transportes_sorteadas)

    print(f"\nVocê categorizou corretamente {acertos_frutas} frutas e {acertos_transportes} transportes.")
    print("Frutas corretas que deveriam estar:", [p for p in palavras_sorteadas if p in categorias["frutas"]])
    print("Transportes corretos que deveriam estar:", [p for p in palavras_sorteadas if p in categorias["transportes"]])

    return {"palavras_apresentadas": palavras_sorteadas,
            "resposta_usuario_frutas": " ".join(resposta_usuario_frutas), # Salva como string para registro
            "resposta_usuario_transportes": " ".join(resposta_usuario_transportes), # Salva como string para registro
            "categorias_corretas": categorias,
            "acertos_frutas": acertos_frutas,
            "acertos_transportes": acertos_transportes,
            "total_palavras_sorteadas": len(palavras_sorteadas),
            "acertou": acertou_totalmente} # Usa a verificação de acerto total

In [28]:
######################################
# Função para registrar o desempenho #
######################################
def registrar_desempenho(usuario, resultado, tipo_exercicio):
    if usuario not in historico_desempenho:
        historico_desempenho[usuario] = []
    historico_desempenho[usuario].append({
        "timestamp": datetime.now().isoformat(),
        "tipo_exercicio": tipo_exercicio,
        **resultado
    })

In [40]:
#####################################################
# Função principal para iniciar a sessão com fases. #
#####################################################
def iniciar_sessao(usuario="usuario_teste", num_interacoes=3, fase_atual=1, pontuacao_fase=0, num_exercicios_fase=0, limiar_acerto=70):
    exercicios_memoria_fase1 = [exercicio_memoria_palavras, exercicio_memoria_numeros, exercicio_memoria_cores]
    exercicios_memoria_fase2 = [exercicio_memoria_palavras, exercicio_memoria_numeros, exercicio_memoria_cores, exercicio_memoria_visual]
    exercicios_memoria_fase3 = [exercicio_memoria_palavras, exercicio_memoria_numeros, exercicio_memoria_cores, exercicio_memoria_visual]

    exercicios_atencao_fase1 = [exercicio_atencao_impar, exercicio_atencao_contagem_letras]
    exercicios_atencao_fase2 = [exercicio_atencao_impar, exercicio_atencao_contagem_letras, exercicio_atencao_simbolos]
    exercicios_atencao_fase3 = [exercicio_atencao_impar, exercicio_atencao_contagem_letras, exercicio_atencao_simbolos, exercicio_atencao_cancelamento]

    exercicios_linguagem_fase1 = [exercicio_linguagem_completar_frase, exercicio_linguagem_sinonimo]
    exercicios_linguagem_fase2 = [exercicio_linguagem_completar_frase, exercicio_linguagem_sinonimo, exercicio_linguagem_anagrama]
    exercicios_linguagem_fase3 = [exercicio_linguagem_completar_frase, exercicio_linguagem_sinonimo, exercicio_linguagem_anagrama, exercicio_linguagem_categorizacao]

    exercicios = {
        1: {"memoria": exercicios_memoria_fase1, "atencao": exercicios_atencao_fase1, "linguagem": exercicios_linguagem_fase1},
        2: {"memoria": exercicios_memoria_fase2, "atencao": exercicios_atencao_fase2, "linguagem": exercicios_linguagem_fase2},
        3: {"memoria": exercicios_memoria_fase3, "atencao": exercicios_atencao_fase3, "linguagem": exercicios_linguagem_fase3}
    }

    if fase_atual > 3:
        print("\nVocê completou todas as fases! Parabéns pelo seu treino cognitivo!")
        return

    print(f"\n--- Iniciando Fase {fase_atual} (Interação {num_exercicios_fase // 3 + 1}-{(num_interacoes * 3) // 3} total de interações) para o usuário: {usuario} ---") # Ajuste na contagem de interações
    pontuacao_interacao = 0
    num_exercicios_interacao = 0

    tipos_exercicios = ["memoria", "atencao", "linguagem"]
    # Garante que todos os tipos de exercício da fase atual sejam tentados em cada interação
    random.shuffle(tipos_exercicios) # Mistura a ordem dos tipos de exercício por interação

    for tipo in tipos_exercicios:
        # Escolhe um exercício aleatório disponível para a fase e tipo
        funcao_exercicio = random.choice(exercicios[fase_atual][tipo])
        print(f"\n--- Exercício de {tipo.capitalize()} (Fase {fase_atual}) ---") # Capitaliza o tipo de exercício para melhor apresentação
        resultado = funcao_exercicio()
        registrar_desempenho(usuario, resultado, tipo)
        if resultado.get("acertou", False):
            pontuacao_interacao += 1
        num_exercicios_interacao += 1

    nova_pontuacao_fase = pontuacao_fase + pontuacao_interacao
    novo_num_exercicios_fase = num_exercicios_fase + num_exercicios_interacao
    taxa_acerto_fase = (nova_pontuacao_fase / novo_num_exercicios_fase) * 100 if novo_num_exercicios_fase > 0 else 0

    print(f"\n--- Fim da Interação ---")
    print(f"Pontuação nesta interação: {pontuacao_interacao} de {num_exercicios_interacao}.")
    print(f"Pontuação total na Fase {fase_atual}: {nova_pontuacao_fase} de {novo_num_exercicios_fase} ({taxa_acerto_fase:.2f}%).")

    # Lógica de progressão de fase
    exercicios_por_fase = num_interacoes * 3
    if novo_num_exercicios_fase < exercicios_por_fase:
        # Continua na mesma fase se ainda não fez o número total de exercícios para a fase
        print(f"Continuando na Fase {fase_atual}. Faltam {exercicios_por_fase - novo_num_exercicios_fase} exercícios para completar a fase.")
        iniciar_sessao(usuario, num_interacoes, fase_atual, nova_pontuacao_fase, novo_num_exercicios_fase, limiar_acerto)
    else:
        # Concluiu o número de exercícios para a fase
        print(f"\nVocê concluiu a Fase {fase_atual} com uma taxa de acerto de {taxa_acerto_fase:.2f}%.")
        if fase_atual < 3 and taxa_acerto_fase >= limiar_acerto: # Critério de avanço de fase (agora usa o limiar)
            proxima_fase = fase_atual + 1
            print(f"Parabéns! Avançando para a Fase {proxima_fase} com exercícios um pouco mais desafiadores!")
            # Reinicia a pontuação e contagem para a próxima fase
            iniciar_sessao(usuario, num_interacoes, proxima_fase, 0, 0, limiar_acerto)
        elif fase_atual == 3 and taxa_acerto_fase >= limiar_acerto:
             mensagem_nivel3 = f"\n🎉 **Parabéns! Você concluiu a Fase 3 com uma taxa de acerto impressionante de {taxa_acerto_fase:.2f}%! **🎉"
             display(HTML(f'<font size="+3">{mensagem_nivel3}</font>'))
             print("Seu desempenho cognitivo está excelente! Continue mantendo sua mente ativa com exercícios regulares.")
        else:
            print(f"Seu desempenho na Fase {fase_atual} foi de {taxa_acerto_fase:.2f}%. Você precisa de pelo menos {limiar_acerto}% de acerto para avançar de fase.")
            if fase_atual < 3:
                print(f"Por favor, repita a Fase {fase_atual} para melhorar sua pontuação.")
                # Repete a fase atual com os mesmos parâmetros iniciais
                iniciar_sessao(usuario, num_interacoes, fase_atual, 0, 0, limiar_acerto)
            else:
                print("Você completou todos os exercícios disponíveis na Fase 3. Continue praticando para melhorar seu desempenho!")

    # --- Feedback Cognitivo (ao final de toda a sessão de num_interacoes * 3 exercícios) ---
    # Este feedback agora aparece apenas no final da série de exercícios definidos por num_interacoes
    if novo_num_exercicios_fase == (num_interacoes * 3) and fase_atual <= 3:
         print(f"\n--- Feedback Cognitivo Final da Fase {fase_atual} ---")
         # Calcula a pontuação e exercícios da fase atual
         pontuacao_fase_atual = sum(item.get("acertou", False) for item in historico_desempenho.get(usuario, []) if item.get("tipo_exercicio") in exercicios[fase_atual].keys()) # Ajuste para pegar apenas os exercícios da fase atual
         num_exercicios_fase_atual = sum(1 for item in historico_desempenho.get(usuario, []) if item.get("tipo_exercicio") in exercicios[fase_atual].keys()) # Ajuste similar
         taxa_acerto_fase_atual = (pontuacao_fase_atual / num_exercicios_fase_atual) * 100 if num_exercicios_fase_atual > 0 else 0

         print(f"Você acertou {pontuacao_fase_atual} de {num_exercicios_fase_atual} exercícios ({taxa_acerto_fase_atual:.2f}% de acerto) na Fase {fase_atual}.")
         if taxa_acerto_fase_atual >= 70:
             print("Seu desempenho cognitivo nesta fase foi bom. Continue assim!")
         elif taxa_acerto_fase_atual >= 50:
             print("Seu desempenho cognitivo nesta fase foi mediano. Pratique para melhorar.")
         else:
             print("Seu desempenho cognitivo nesta fase precisa de mais atenção. Tente focar e praticar mais.")

         if fase_atual == 3:
             # Feedback cognitivo geral ao final da Fase 3
             pontuacao_sessao_total = sum(item.get("acertou", False) for usuario_hist in historico_desempenho.values() for item in usuario_hist)
             num_exercicios_sessao_total = sum(len(usuario_hist) for usuario_hist in historico_desempenho.values())
             taxa_acerto_sessao_total = (pontuacao_sessao_total / num_exercicios_sessao_total) * 100 if num_exercicios_sessao_total > 0 else 0

             mensagem_cog_final = f"\n--- Feedback Cognitivo Final da Sessão ---\n"
             display(HTML(f'<font size="+2">{mensagem_cog_final}</font>'))
             display(to_markdown(f"**Ao longo da sessão, você acertou {pontuacao_sessao_total} de {num_exercicios_sessao_total} exercícios ({taxa_acerto_sessao_total:.2f}% de acerto).**"))
             #print(f"Ao longo da sessão, você acertou {pontuacao_sessao_total} de {num_exercicios_sessao_total} exercícios ({taxa_acerto_sessao_total:.2f}% de acerto).")

             if taxa_acerto_sessao_total >= 70:
                 print("**Seu desempenho cognitivo geral é bom!**")
             elif taxa_acerto_sessao_total >= 50:
                 print("**Seu desempenho cognitivo geral está na média.**")
             else:
                 print("**Seu desempenho cognitivo geral pode melhorar com mais prática.**")
             print("\nHistórico de Desempenho Completo:")
             print(historico_desempenho.get(usuario, "Nenhum desempenho registrado."))


# Limpar o histórico para uma nova sessão com fases
historico_desempenho = {}
# Iniciar a sessão com progressão de fases
# num_interacoes agora define o número total de interações em toda a sessão (num_interacoes * 3 exercícios no total)
iniciar_sessao(usuario=usuario_atual, num_interacoes=2, limiar_acerto=70) # Exemplo com um total de 2 interações * 3 exercícios = 6 exercícios na sessão. Adicionado o limiar como argumento.


--- Iniciando Fase 1 (Interação 1-2 total de interações) para o usuário: usuario_teste ---

--- Exercício de Linguagem (Fase 1) ---

Qual o sinônimo da palavra 'pequeno'?
1. diminuto
2. minúsculo
3. rápido
4. liliputiano
Digite o número da opção correta: 2
Resposta incorreta. O sinônimo correto era: liliputiano.

--- Exercício de Memoria (Fase 1) ---

Memorize a seguinte sequência de 6 números em 5 segundos:
[2, 3, 0, 0, 2, 8]

Tempo esgotado!
Digite a sequência de números que você lembra, separada por espaço: 2 3 0 0 2 8
Parabéns! Você acertou a sequência.

--- Exercício de Atencao (Fase 1) ---

Encontre o número ímpar na seguinte sequência:
[2, 2, 1, 4, 4, 4, 8]
Digite o número ímpar encontrado: 2 2 1 4 4 8
Resposta incorreta. O número ímpar era 1.

--- Fim da Interação ---
Pontuação nesta interação: 1 de 3.
Pontuação total na Fase 1: 1 de 3 (33.33%).
Continuando na Fase 1. Faltam 3 exercícios para completar a fase.

--- Iniciando Fase 1 (Interação 2-2 total de interações) para o us

Seu desempenho cognitivo está excelente! Continue mantendo sua mente ativa com exercícios regulares.

--- Feedback Cognitivo Final da Fase 3 ---
Você acertou 28 de 36 exercícios (77.78% de acerto) na Fase 3.
Seu desempenho cognitivo nesta fase foi bom. Continue assim!


> **Ao longo da sessão, você acertou 28 de 36 exercícios (77.78% de acerto).**

**Seu desempenho cognitivo geral é bom!**

Histórico de Desempenho Completo:
[{'timestamp': '2025-05-17T03:19:54.782848', 'tipo_exercicio': 'linguagem', 'palavra_chave': 'pequeno', 'sinonimo_correto': 'liliputiano', 'opcoes': ['diminuto', 'minúsculo', 'rápido', 'liliputiano'], 'resposta_usuario': 'minúsculo', 'acertou': False}, {'timestamp': '2025-05-17T03:20:10.817492', 'tipo_exercicio': 'memoria', 'sequencia_apresentada': [2, 3, 0, 0, 2, 8], 'resposta_usuario': '2 3 0 0 2 8', 'acertou': True}, {'timestamp': '2025-05-17T03:22:34.520440', 'tipo_exercicio': 'atencao', 'sequencia_apresentada': [2, 2, 1, 4, 4, 4, 8], 'resposta_usuario': '2 2 1 4 4 8', 'numero_impar_correto': 1, 'acertou': False}, {'timestamp': '2025-05-17T03:22:37.851394', 'tipo_exercicio': 'linguagem', 'palavra_chave': 'pequeno', 'sinonimo_correto': 'minúsculo', 'opcoes': ['diminuto', 'rápido', 'liliputiano', 'minúsculo'], 'resposta_usuario': 'liliputiano', 'acertou': False}, {'timestamp': '2025-05-17T03:22:47.446227', '